In [1]:
!python -V

Python 3.11.2


In [1]:
import pandas as pd

In [6]:
import pickle

In [7]:
import seaborn as sns
import matplotlib.pyplot as plt

In [8]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [9]:
import mlflow


mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location=('file:///c:/Users/ulugb/Documents/Personal/Exercises/6. MLOps '
 'Zoomcamp/02-experiment-tracking/mlruns/1'), creation_time=1685203684433, experiment_id='1', last_update_time=1685203684433, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [11]:
def read_dataframe(filename):
    # df = pd.read_csv(filename)
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [12]:
# df_train = read_dataframe('./data/green_tripdata_2021-01.csv')
# df_val = read_dataframe('./data/green_tripdata_2021-02.csv')

df_train = read_dataframe('./homework/green_tripdata_2022-01.parquet')
df_val = read_dataframe('./homework/green_tripdata_2022-02.parquet')

In [13]:
len(df_train), len(df_val)

(59603, 66097)

In [15]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [16]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [17]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [18]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

6.928931230104186

In [24]:
with open('./models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [23]:
# import os
# os.getcwd()

'c:\\Users\\ulugb\\Documents\\Personal\\Exercises\\6. MLOps Zoomcamp\\02-experiment-tracking'

In [25]:
with mlflow.start_run():

    # mlflow.set_tag("developer", "cristian")
    mlflow.set_tag("developer", "UD")

    # mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.csv")
    # mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.csv")
    
    mlflow.log_param("train-data-path", "./homework/green_tripdata_2022-01.parquet")
    mlflow.log_param("valid-data-path", "./homework/green_tripdata_2022-02.parquet")

    alpha = 0.1
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    mlflow.log_artifact(local_path="./models/lin_reg.bin", artifact_path="models_pickle")

In [26]:
import xgboost as xgb

In [27]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [28]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [37]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred_train = booster.predict(train)
        y_pred = booster.predict(valid)
        
        rmse_train = mean_squared_error(y_train, y_pred_train, squared=False)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        
        mlflow.log_metric("train rmse", rmse_train)
        mlflow.log_metric("val rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [39]:
search_space = {
    'tree_method': 'auto',
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

[14:26:03] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07593ffd91cd9da33-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:9.53054                           
[1]	validation-rmse:6.96681                           
[2]	validation-rmse:6.26718                           
[3]	validation-rmse:6.07145                           
[4]	validation-rmse:5.99041                           
[5]	validation-rmse:5.95548                           
[6]	validation-rmse:5.94088                           
[7]	validation-rmse:5.93216                           
[8]	validation-rmse:5.92868                           
[9]	validation-rmse:5.92483                           
[10]	validation-rmse:5.91316                          
[11]	validation-rmse:5.91264                          
[12]	validation-rmse:5.91406                          
[13]	validation-rmse:5.91469                          

In [ ]:
# best parameters
"""learning_rate	0.320562081697366
max_depth	6
min_child_weight	1.6995177751824728
objective	reg:linear
reg_alpha	0.3542529659352334
reg_lambda	0.0027076559333701193
seed	42
tree_method	auto
"""

In [ ]:
mlflow.xgboost.autolog(disable=True)

In [40]:
with mlflow.start_run():
    
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)

    best_params = {
        'learning_rate': 0.320562081697366,
        'max_depth': 6,
        'min_child_weight': 1.6995177751824728,
        'objective': 'reg:linear',
        'reg_alpha': 0.3542529659352334,
        'reg_lambda': 0.002707655933370119,
        'seed': 42,
        'tree_method': 'auto'
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )

    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    # mlflow.log_metric("rmse", rmse)
    mlflow.log_metric("val rmse", rmse)

    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
        
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

[14:59:30] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07593ffd91cd9da33-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:12.91894
[1]	validation-rmse:10.07367
[2]	validation-rmse:8.35830
[3]	validation-rmse:7.36984
[4]	validation-rmse:6.81284
[5]	validation-rmse:6.50730
[6]	validation-rmse:6.33228
[7]	validation-rmse:6.23112
[8]	validation-rmse:6.17212
[9]	validation-rmse:6.13429
[10]	validation-rmse:6.11382
[11]	validation-rmse:6.09649
[12]	validation-rmse:6.08833
[13]	validation-rmse:6.08114
[14]	validation-rmse:6.07729
[15]	validation-rmse:6.07504
[16]	validation-rmse:6.07156
[17]	validation-rmse:6.06824
[18]	validation-rmse:6.06429
[19]	validation-rmse:6.05680
[20]	validation-rmse:6.05375
[21]	validation-rmse:6.05121
[22]	validation-rmse:6.04850
[23]	validation-rmse:6.04521
[24]	validation-rmse:6.04275
[25]	validation-rmse:6.04190
[26]	validation-rmse:6.

c:\Users\ulugb\AppData\Local\Programs\Python\Python311\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [ ]:
# model registry

In [44]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR

mlflow.sklearn.autolog()

# for model_class in (RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, LinearSVR):
for model_class in (GradientBoostingRegressor, LinearSVR):

    with mlflow.start_run():

        # mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.csv")
        # mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.csv")
        mlflow.log_param("train-data-path", "./homework/green_tripdata_2022-01.parquet")
        mlflow.log_param("valid-data-path", "./homework/green_tripdata_2022-02.parquet")
        mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

        mlmodel = model_class()
        mlmodel.fit(X_train, y_train)

        y_pred = mlmodel.predict(X_val)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("val rmse", rmse)
        

c:\Users\ulugb\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
